# 0 Create the Cognite Client

In [1]:
import sys

sys.path.append('..')
from utils.auth import create_cognite_client

client = create_cognite_client()

Opening in existing browser session.


libva error: vaGetDriverNameByIndex() failed with unknown libva error, driver_name = (null)
/home/keepfloyding/.local/lib/python3.10/site-packages/cognite/client/_cognite_client.py:84: UserWarning: You are using version 2.55.0 of the SDK, however version 2.56.1 is available. Upgrade or set the environment variable 'COGNITE_DISABLE_PYPI_VERSION_CHECK' to suppress this warning.
  self._config = ClientConfig(


# 1 Create a data set

In [3]:
from cognite.client.data_classes import DataSet

my_data_set = DataSet(external_id = 'world_info_jack', name = 'world_info_jack', write_protected=True)

created_data_set = client.data_sets.create(my_data_set)

data_set_id = created_data_set.id

print(data_set_id)

6133408726791096


# 2 Create Asset Hierarchy

In [5]:
from cognite.client.data_classes import Asset

# Create Root Asset
root_asset = client.assets.create(Asset(name='world', data_set_id=data_set_id))
root_asset

,value
name,world
data_set_id,6133408726791096
id,4814454518239648
created_time,1659027243120
last_updated_time,1659027243120
root_id,4814454518239648


In [6]:
from cognite.client.data_classes import AssetUpdate

# Update the name of the root asset
my_update = AssetUpdate(id=root_asset.id).name.set("global")
client.assets.update(my_update)

,value
name,global
data_set_id,6133408726791096
id,4814454518239648
created_time,1659027243120
last_updated_time,1659027251568
root_id,4814454518239648


In [8]:
import pandas as pd

# Read country data and see all unique regions
df = pd.read_csv('../data/all_countries.csv')
df['region'].unique()

array(['Asia', 'Europe', 'Africa', 'Oceania', 'Americas', 'None'],
      dtype=object)

In [13]:
# Create each region as a new asset
store = []
for region in df['region'].unique():
    asset=Asset(name=region, data_set_id=data_set_id,parent_id=root_asset.id)
    store.append(asset)

client.assets.create(store)

,name,parent_id,data_set_id,metadata,id,created_time,last_updated_time,root_id
0,Asia,4814454518239648,6133408726791096,{},3956719264398039,1659027477151,1659027477151,4814454518239648
1,Europe,4814454518239648,6133408726791096,{},4850791345326164,1659027477151,1659027477151,4814454518239648
2,Africa,4814454518239648,6133408726791096,{},7860216995992688,1659027477151,1659027477151,4814454518239648
3,Oceania,4814454518239648,6133408726791096,{},8988177472119253,1659027477151,1659027477151,4814454518239648
4,Americas,4814454518239648,6133408726791096,{},7039848497891817,1659027477151,1659027477151,4814454518239648
5,None,4814454518239648,6133408726791096,{},5826238180166344,1659027477151,1659027477151,4814454518239648


In [17]:
# Find the mapping between the region level asset and its id
region_to_id = client.assets.list(data_set_ids=[data_set_id]).to_pandas()[['name','id']].set_index('name')['id'].to_dict()
region_to_id

{'Africa': 7860216995992688,
 'world': 990674513871418,
 'Asia': 3956719264398039,
 'Americas': 7039848497891817,
 'Oceania': 8988177472119253,
 'global': 4814454518239648,
 'Europe': 7957777709733657,
 'None': 8367044107042036}

In [18]:
# Find all countries and which regions they belong to
countries_to_regions = df[['name','region']].set_index('name').to_dict()['region']
countries_to_regions

{'Afghanistan': 'Asia',
 'Åland Islands': 'Europe',
 'Albania': 'Europe',
 'Algeria': 'Africa',
 'American Samoa': 'Oceania',
 'Andorra': 'Europe',
 'Angola': 'Africa',
 'Anguilla': 'Americas',
 'Antarctica': 'None',
 'Antigua and Barbuda': 'Americas',
 'Argentina': 'Americas',
 'Armenia': 'Asia',
 'Aruba': 'Americas',
 'Australia': 'Oceania',
 'Austria': 'Europe',
 'Azerbaijan': 'Asia',
 'Bahamas': 'Americas',
 'Bahrain': 'Asia',
 'Bangladesh': 'Asia',
 'Barbados': 'Americas',
 'Belarus': 'Europe',
 'Belgium': 'Europe',
 'Belize': 'Americas',
 'Benin': 'Africa',
 'Bermuda': 'Americas',
 'Bhutan': 'Asia',
 'Bolivia (Plurinational State of)': 'Americas',
 'Bonaire, Sint Eustatius and Saba': 'Americas',
 'Bosnia and Herzegovina': 'Europe',
 'Botswana': 'Africa',
 'Bouvet Island': 'Americas',
 'Brazil': 'Americas',
 'British Indian Ocean Territory': 'Africa',
 'Brunei Darussalam': 'Asia',
 'Bulgaria': 'Europe',
 'Burkina Faso': 'Africa',
 'Burundi': 'Africa',
 'Cabo Verde': 'Africa',
 'Ca

In [19]:
# Create an asset for each country
store = []
for country in countries_to_regions:
    
    region = countries_to_regions[country]
    asset=Asset(name=country, data_set_id=data_set_id,parent_id=region_to_id[region])
    store.append(asset)
    
client.assets.create(store)

In [21]:
# Find all assets in the Europe region
client.assets.list(data_set_ids=[data_set_id],parent_ids=[region_to_id['Europe']],limit=-1).to_pandas()

,name,parentId,dataSetId,metadata,id,createdTime,lastUpdatedTime,rootId
0,"Moldova, Republic of",7957777709733657,6133408726791096,{},6887440463175,1659027641778,1659027641778,4814454518239648
1,Svalbard and Jan Mayen,7957777709733657,6133408726791096,{},698806610782864,1659027647310,1659027647310,4814454518239648
2,Norway,7957777709733657,6133408726791096,{},705530469890494,1659027643397,1659027643397,4814454518239648
3,Malta,7957777709733657,6133408726791096,{},882307798078568,1659027641134,1659027641134,4814454518239648
4,Slovakia,7957777709733657,6133408726791096,{},918294277087161,1659027646376,1659027646376,4814454518239648
5,Bulgaria,7957777709733657,6133408726791096,{},982652123952713,1659027633568,1659027633568,4814454518239648
6,Hungary,7957777709733657,6133408726791096,{},996666741032903,1659027638471,1659027638471,4814454518239648
7,Croatia,7957777709733657,6133408726791096,{},1202254073563557,1659027635028,1659027635028,4814454518239648
8,Slovenia,7957777709733657,6133408726791096,{},1236641090165021,1659027646489,1659027646489,4814454518239648
9,Jersey,7957777709733657,6133408726791096,{},1487206610269610,1659027639390,1659027639390,4814454518239648


# 3 Adding Time Series Data

In [32]:
# Read population data
df= pd.read_csv('../data/populations_postprocessed.csv', index_col=0)
df.head()

,Aruba,Afghanistan,Angola,Albania,Andorra,United Arab Emirates,Argentina,Armenia,American Samoa,Antigua and Barbuda,...,Uganda,Ukraine,Uruguay,Uzbekistan,Virgin Islands (U.S.),Vanuatu,Samoa,South Africa,Zambia,Zimbabwe
1990-01-01,62152.0,12412311.0,11848385.0,3286542.0,54508.0,1828437.0,32618648.0,3538164.0,47351.0,62533.0,...,17354395.0,51891400.0,3109598.0,20510000.0,103963.0,146575.0,162797.0,36800507.0,8036849.0,10432409.0
1991-01-01,64623.0,13299016.0,12248901.0,3266790.0,56666.0,1937159.0,33079002.0,3505249.0,48682.0,63363.0,...,17953534.0,52000500.0,3131657.0,20952000.0,104807.0,150718.0,164000.0,37718952.0,8246662.0,10681008.0
1992-01-01,68240.0,14485543.0,12657361.0,3247039.0,58882.0,2052892.0,33529320.0,3442820.0,49900.0,64459.0,...,18561668.0,52150400.0,3154459.0,21449000.0,105712.0,155176.0,165490.0,38672611.0,8451346.0,10900511.0
1993-01-01,72495.0,15816601.0,13075044.0,3227287.0,60974.0,2173135.0,33970103.0,3363111.0,51025.0,65777.0,...,19175986.0,52179200.0,3177734.0,21942000.0,106578.0,159743.0,167117.0,39633754.0,8656484.0,11092775.0
1994-01-01,76705.0,17075728.0,13503753.0,3207536.0,62676.0,2294377.0,34402669.0,3283664.0,52099.0,67201.0,...,19793541.0,51921400.0,3201149.0,22377000.0,107318.0,164128.0,168689.0,40564061.0,8869745.0,11261752.0


In [25]:
# Find mapping of country to its id
country_to_id = client.assets.list(data_set_ids=[data_set_id], limit=-1).to_pandas()[['name','id']].set_index('name')['id'].to_dict()

country_to_id

{'Moldova, Republic of': 6887440463175,
 'Mauritius': 24245784711959,
 'Kenya': 118043905096570,
 'Thailand': 144564637701025,
 'Tonga': 161447343836851,
 'Africa': 7860216995992688,
 'Botswana': 214830130247617,
 'Azerbaijan': 220740559462349,
 'Tunisia': 233460106802571,
 'Burkina Faso': 272119597883532,
 'Lesotho': 294571079576969,
 'Georgia': 331705715305538,
 'Ethiopia': 528038785850175,
 'Bhutan': 629634915222618,
 'Grenada': 672828489854230,
 'Svalbard and Jan Mayen': 698806610782864,
 'Norway': 705530469890494,
 'Canada': 776249613319493,
 'Fiji': 810702146085903,
 'Western Sahara': 837959032873187,
 'Cocos (Keeling) Islands': 877582077071674,
 "Lao People's Democratic Republic": 881853517011061,
 'Malta': 882307798078568,
 'Guam': 913610326884313,
 'Eswatini': 914118173071515,
 'Slovakia': 918294277087161,
 'Uruguay': 942141643860658,
 'Bulgaria': 982652123952713,
 'world': 990674513871418,
 'Hungary': 996666741032903,
 'French Guiana': 1007190471634089,
 'Kuwait': 10559653414

In [26]:
from cognite.client.data_classes import TimeSeries

# Create a time series object for each country to store population data
store = []
for country in df.columns:
    ts = TimeSeries(name=f'{country}_population',data_set_id=data_set_id, asset_id=country_to_id[country])
    store.append(ts)
    
client.time_series.create(store)

,id,name,is_string,metadata,asset_id,is_step,security_categories,data_set_id,created_time,last_updated_time
0,1743611819981898,Aruba_population,False,{},3132599988801875,False,[],6133408726791096,1659027953868,1659027953868
1,1076467877030158,Afghanistan_population,False,{},2356719482572089,False,[],6133408726791096,1659027953868,1659027953868
2,7287895875443546,Angola_population,False,{},1276279265438934,False,[],6133408726791096,1659027953868,1659027953868
3,3051160058271065,Albania_population,False,{},8462051300313348,False,[],6133408726791096,1659027953868,1659027953868
4,8119193982373629,Andorra_population,False,{},6379142196133863,False,[],6133408726791096,1659027953868,1659027953868
...,...,...,...,...,...,...,...,...,...,...
178,501265599905144,Vanuatu_population,False,{},6532609186995507,False,[],6133408726791096,1659027953868,1659027953868
179,8683116680963244,Samoa_population,False,{},8929267941830694,False,[],6133408726791096,1659027953868,1659027953868
180,1559936261890537,South Africa_population,False,{},1808591083555574,False,[],6133408726791096,1659027953868,1659027953868
181,4157164586730559,Zambia_population,False,{},8637004914279868,False,[],6133408726791096,1659027953868,1659027953868


In [33]:
# Change the name of the columns in the data frame
df.columns = df.columns+'_population'

# Find the mapping of the time series to its id
ts_to_id = client.time_series.list(data_set_ids=[data_set_id], limit=-1).to_pandas()[['name','id']].set_index('name')['id'].to_dict()

# Rename data frame so that columns correspond to the time seris id
df = df.rename(columns=ts_to_id)

df.head()

,1743611819981898,1076467877030158,7287895875443546,3051160058271065,8119193982373629,6441871508656048,3799878974691437,2573850347832443,3951589151203550,4801880051065120,...,5197352968019859,7104276752368489,7663422770659640,3491721636262267,4255762038472505,501265599905144,8683116680963244,1559936261890537,4157164586730559,7970629359618913
1990-01-01,62152.0,12412311.0,11848385.0,3286542.0,54508.0,1828437.0,32618648.0,3538164.0,47351.0,62533.0,...,17354395.0,51891400.0,3109598.0,20510000.0,103963.0,146575.0,162797.0,36800507.0,8036849.0,10432409.0
1991-01-01,64623.0,13299016.0,12248901.0,3266790.0,56666.0,1937159.0,33079002.0,3505249.0,48682.0,63363.0,...,17953534.0,52000500.0,3131657.0,20952000.0,104807.0,150718.0,164000.0,37718952.0,8246662.0,10681008.0
1992-01-01,68240.0,14485543.0,12657361.0,3247039.0,58882.0,2052892.0,33529320.0,3442820.0,49900.0,64459.0,...,18561668.0,52150400.0,3154459.0,21449000.0,105712.0,155176.0,165490.0,38672611.0,8451346.0,10900511.0
1993-01-01,72495.0,15816601.0,13075044.0,3227287.0,60974.0,2173135.0,33970103.0,3363111.0,51025.0,65777.0,...,19175986.0,52179200.0,3177734.0,21942000.0,106578.0,159743.0,167117.0,39633754.0,8656484.0,11092775.0
1994-01-01,76705.0,17075728.0,13503753.0,3207536.0,62676.0,2294377.0,34402669.0,3283664.0,52099.0,67201.0,...,19793541.0,51921400.0,3201149.0,22377000.0,107318.0,164128.0,168689.0,40564061.0,8869745.0,11261752.0


In [34]:
# Insert the population data as a dataframe
client.datapoints.insert_dataframe(df, dropna=True)

In [35]:
# Find the latest value for the population of Benin
client.datapoints.retrieve_latest(ts_to_id['Benin_population'])

,3609515985995211
2021-01-01,12451031.0


In [36]:
# List all assets in Europe
assets = client.assets.list(data_set_ids=[data_set_id],parent_ids=[region_to_id['Europe']], limit=-1)

# List all time series of those assets
ts = client.time_series.list(data_set_ids=[data_set_id],asset_ids=[asset.id for asset in assets], limit=-1)

# Retrieve the latest data for all of these time series
data = client.datapoints.retrieve_latest([item.id for item in ts]).to_pandas()

data.head()

,341243902417193,738229187761245,1696930254138174,1842254717574791,2183379464251574,2230258722670301,2494468767063885,2640005654034677,2658229713050040,2871464963686590,...,6833942010842777,7070682077501629,7104276752368489,7482265016795204,7497066200430994,7979330735605335,8013998572874809,8119193982373629,8367431265434749,8376694158838264
2021-01-01,67499343.0,47326687.0,1883162.0,8956279.0,2065092.0,38254.0,10664568.0,17533405.0,1329254.0,49053.0,...,10299423.0,34010.0,43814581.0,6844078.0,143446060.0,11587882.0,10415811.0,77354.0,8697723.0,39520.0


In [38]:
# Perform a sum to evaluate whole population
data.T.sum()

2021-01-01    656621484.0
dtype: float64

## 4. Uploading Files

In [39]:
import os

# List all files and upload them
for item in os.listdir('../data/files'):
    country = item.split('.')[0]
    client.files.upload(f"../data/files/{item}", name=f"{country}_data_sheet",data_set_id=data_set_id, asset_ids=[country_to_id[country]])

In [40]:
# List all files related to Vanuatu
client.files.list(asset_ids=[country_to_id['Vanuatu']])

,name,asset_ids,data_set_id,id,uploaded,uploaded_time,created_time,last_updated_time
0,Vanuatu_data_sheet,[6532609186995507],6133408726791096,6354826719252420,True,1659028402790,1659028402316,1659028402316


## 5. Adding events

In [55]:
# Read event data
df = pd.read_csv('../data/events.csv')
df.head()

,Dis No,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Event Name,Country,ISO,Region,Continent,Location,Origin,Start Year,Start Month,Start Day,End Year,End Month,End Day
0,2011-0159-ESP,Natural,Geophysical,Earthquake,Ground movement,NaN,Spain,ESP,Southern Europe,Europe,"Lorca municipality (Murcia district, Region de...",NaN,2011,5,11,2011,5,11
1,2010-0170-ISL,Natural,Geophysical,Volcanic activity,Ash fall,Eyjafjallajokull,Iceland,ISL,Northern Europe,Europe,"Fljotshlid, Eyjafjöll, and Landeyjar villages...",NaN,2010,3,1,2010,4,28
2,2010-0574-SRB,Natural,Geophysical,Earthquake,Ground movement,NaN,Serbia,SRB,Southern Europe,Europe,Kraljevo town (Raski Province),NaN,2010,11,3,2010,11,3
3,2012-0152-BGR,Natural,Geophysical,Earthquake,Ground movement,NaN,Bulgaria,BGR,Eastern Europe,Europe,"Pernik town (Pernik Province), Sofia town (Sof...",NaN,2012,5,22,2012,5,22
4,2013-0121-HUN,Natural,Geophysical,Earthquake,Ground movement,NaN,Hungary,HUN,Eastern Europe,Europe,Heves province,NaN,2013,4,14,2013,4,23


In [56]:
from cognite.client.data_classes import Event


store = []
# Cycle through each row
for _,row in df.iterrows():
    
    # Unpack the row
    _,_,_,Disaster_Type,Disaster_Subtype,_,Country,_,_,_,Location,_,Start_Year,Start_Month,Start_Day,End_Year,End_Month,End_Day = row
        
    # Create start and end datetimes
    start_date = pd.to_datetime(f'{Start_Year}-{Start_Month}-{int(Start_Day)}')
    end_date = pd.to_datetime(f'{End_Year}-{End_Month}-{int(End_Day)}')
    
    # Create metadata
    metadata = {'Location':Location}
    
    # Create the event and store
    event = Event(start_time=int(start_date.timestamp()*1000), end_time=int(end_date.timestamp()*1000),metadata=metadata,data_set_id=data_set_id, type=Disaster_Type, subtype=Disaster_Subtype, asset_ids=[country_to_id[Country]])
    store.append(event)

client.events.create(store)

,data_set_id,start_time,end_time,type,subtype,metadata,asset_ids,id,last_updated_time,created_time
0,6133408726791096,1305072000000,1305072000000,Earthquake,Ground movement,{'Location': 'Lorca municipality (Murcia distr...,[8752177333776915],1014837573416752,1659028943134,1659028943134
1,6133408726791096,1267401600000,1272412800000,Volcanic activity,Ash fall,"{'Location': ' Fljotshlid, Eyjafjöll, and Land...",[6169776311653714],8819005418381520,1659028943134,1659028943134
2,6133408726791096,1288742400000,1288742400000,Earthquake,Ground movement,{'Location': 'Kraljevo town (Raski Province)'},[2725709788151657],3194129737744035,1659028943134,1659028943134
3,6133408726791096,1337644800000,1337644800000,Earthquake,Ground movement,"{'Location': 'Pernik town (Pernik Province), S...",[982652123952713],3668593465793278,1659028943134,1659028943134
4,6133408726791096,1365897600000,1366675200000,Earthquake,Ground movement,{'Location': 'Heves province'},[996666741032903],8462767060927079,1659028943134,1659028943134
5,6133408726791096,1337472000000,1337472000000,Earthquake,Ground movement,"{'Location': 'Finale Emilia, Mirandola towns (...",[8466638942579644],5654328226396328,1659028943134,1659028943134
6,6133408726791096,1338249600000,1338249600000,Earthquake,Ground movement,{'Location': 'Modena Province (Emilia-romagna ...,[8466638942579644],6634939121740276,1659028943134,1659028943134
7,6133408726791096,1390694400000,1391385600000,Earthquake,Ground movement,"{'Location': 'Lixouri, Kounopetra, Fiskardo vi...",[5007573311330472],7078925368236568,1659028943134,1659028943134
8,6133408726791096,1400889600000,1400889600000,Earthquake,Ground movement,"{'Location': 'Limnos Island (Lesvou district, ...",[5007573311330472],3320542769078881,1659028943134,1659028943134
9,6133408726791096,1497225600000,1497225600000,Earthquake,Ground movement,"{'Location': 'Vrisa, Plomarion,Plagias, Chios,...",[5007573311330472],4059082217839916,1659028943134,1659028943134


In [57]:
client.events.list(data_set_ids=[data_set_id], type='Volcanic activity')

,data_set_id,start_time,end_time,type,subtype,metadata,asset_ids,id,last_updated_time,created_time
0,6133408726791096,1267401600000,1272412800000,Volcanic activity,Ash fall,"{'Location': ' Fljotshlid, Eyjafjöll, and Land...",[6169776311653714],6104514822595503,1659028849179,1659028849179
1,6133408726791096,1267401600000,1272412800000,Volcanic activity,Ash fall,"{'Location': ' Fljotshlid, Eyjafjöll, and Land...",[6169776311653714],7388065635077550,1659028795510,1659028795510
2,6133408726791096,1267401600000,1272412800000,Volcanic activity,Ash fall,"{'Location': ' Fljotshlid, Eyjafjöll, and Land...",[6169776311653714],8819005418381520,1659028943134,1659028943134
3,6133408726791096,1545609600000,1545782400000,Volcanic activity,None,"{'Location': 'Acicatena, Acireale, Aci Sant’An...",[8466638942579644],2158354877815493,1659028943134,1659028943134


## 6. Add Labels

In [58]:
client.labels.list()

,external_id,name,created_time,data_set_id
0,cold-countries,Cold Climate Countries,1658795787302,4603840493329850
1,REGION,REGION,1657141671698,<NA>
2,COUNTRY,COUNTRY,1657141671698,<NA>
3,BALTIC,BALTIC,1657143421635,<NA>
4,hot-countries,Hot Climate Countries,1658795787302,4603840493329850
5,NORDIC,NORDIC,1657142354433,<NA>


In [59]:
for country in ['Norway','Canada','Latvia']:

    my_update = AssetUpdate(id=country_to_id[country]).labels.set('cold-countries')

    client.assets.update(my_update)

In [60]:
for country in ['Ghana','Qatar','Malaysia']:

    my_update = AssetUpdate(id=country_to_id[country]).labels.set('hot-countries')

    client.assets.update(my_update)

In [61]:
from cognite.client.data_classes import LabelFilter

my_filter = LabelFilter(contains_any=['cold-countries'])

client.assets.list(labels=my_filter,data_set_ids=[data_set_id])

,name,parent_id,data_set_id,metadata,labels,id,created_time,last_updated_time,root_id
0,Norway,7957777709733657,6133408726791096,{},[{'externalId': 'cold-countries'}],705530469890494,1659027643397,1659029038797,4814454518239648
1,Canada,7039848497891817,6133408726791096,{},[{'externalId': 'cold-countries'}],776249613319493,1659027634016,1659029038943,4814454518239648
2,Latvia,7957777709733657,6133408726791096,{},[{'externalId': 'cold-countries'}],4309592236466757,1659027640119,1659029039120,4814454518239648


## 7. Final Task:  Clean-Up and Delete the Data in your Data Set


In [62]:
# Assets
all_assets = [asset.id for asset in client.assets.list(limit=-1, data_set_ids=[data_set_id])]
client.assets.delete(all_assets)

# Time series
all_time_series = [ts.id for ts in client.time_series.list(limit=-1, data_set_ids=[data_set_id])]
client.time_series.delete(all_time_series)

# Files
all_files = [file.id for file in client.files.list(limit=-1, data_set_ids=[data_set_id])]
client.files.delete(all_files)

# Events
all_events = [event.id for event in client.events.list(limit=-1, data_set_ids=[data_set_id])]
client.events.delete(all_events)

print('Success!')

Success!
